## Grabcat - Foreground/background segmentation of cats in images

In [1]:
import numpy as np
import math
import cv2 as cv
import os
import glob
import time
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import scipy.io
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from sklearn.metrics import precision_score,recall_score,accuracy_score

In [2]:
def show_images(image_1, image_2=None, image_3=None, name_1='image_1', name_2='image_2', name_3='image_3', time_out=0):
    """
    Show images received as parameters.
    """
    cv.imshow(name_1, image_1)
    if image_2 is not None:
        cv.imshow(name_2, image_2)
    if image_3 is not None:
        cv.imshow(name_3, image_3)
    cv.waitKey(time_out)
    cv.destroyAllWindows()

In [3]:
def measure_performance(result, gt):
    """
    result : segmentation obtained
    gt: ground-truth segmentation
    returns precision and recall
    """
    result_converted=np.asarray(result)
    print(result_converted)
    result_converted[result_converted<200]=0
    result_converted[result_converted>200]=1
    result_converted=result_converted.flatten()
    gt_converted=np.asarray(gt)
    gt_converted[gt_converted<200]=0
    gt_converted[gt_converted>200]=1
    gt_converted=gt_converted.flatten()
    return precision_score(gt_converted,result_converted),recall_score(gt_converted,result_converted)

In [4]:
def get_regions(image):
    """
    image: image for segmentation
    -select the ROI
    -create list with background pixels
    -create list with foreground pixels
    returns the coordinates of region of interest, cropped_image, background and foreground pixels
    """
    r = cv.selectROI("select the area", image)
    cropped_image = image[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
    print(cropped_image.shape)
    show_images(cropped_image)
    background=[]
    #left
    for i in range(image.shape[0]):
        for j in range(0, r[0]):
            background.append(image[i, j, :])
    #right
    for i in range(image.shape[0]):
        for j in range(r[0] + r[2], image.shape[1]):
            background.append(image[i, j, :])
    #up
    for i in range(r[1]):
        for j in range(r[0], r[0] + r[2]):
            background.append(image[i, j, :])
    #down
    for i in range(r[1] + r[3], image.shape[0]):
        for j in range(r[0], r[0] + r[2]):
            background.append(image[i, j, :])
    b=np.array(background)
    fb=cropped_image.reshape((-1, 3))
    return r,b,fb,cropped_image

In [5]:
def get_kmean_centroids(data, k):
    """
    data: data to be clustered
    k: number of clusters
    returns the center of clusters
    """
    kmeans = KMeans(init="random", n_clusters=k, n_init=10, max_iter=300, random_state=42)
    kmeans.fit(np.float64(data))
    return kmeans.cluster_centers_

In [6]:
def segment_image(cropped_image,iterations,centroids_b,centroids_f,b_orig,fb_orig,img_name):
    """
    cropped_image: region to be segmented
    iterations: number of iterations
    centroids_b: number of centroids for background
    centroids_f: number of centroids for foreground
    b_orig: original pixels for background
    fb_orig: original pixels from foreground and background region
    img_name: image name for segmented image
    -cluster background pixels
    -cluster pixels from foreground and background region
    -compute euclidean distance between pixels from cropped image and cluster centers
    -assign pixels to background if minimum distance is found between a cluster from backgrpund
    returns the segmented image
    """
    b=b_orig.copy()
    fb=fb_orig.copy()
    for cnt in range(iterations):
        kmeans_b=get_kmean_centroids(b,centroids_b)
        kmeans_fb=get_kmean_centroids(fb,centroids_f)
        img_mask=np.zeros((cropped_image.shape[0],cropped_image.shape[1]))
        img_mask.fill(255)
        b=b_orig.copy()
        fb=fb_orig.copy()
        for i in range(cropped_image.shape[0]):
            for j in range(cropped_image.shape[1]):
                distances_b  = []
                distances_fb = []

                for c in kmeans_b:
                    distances_b.append(math.dist(cropped_image[i, j, :], c))
                for c in kmeans_fb:
                    distances_fb.append(math.dist(cropped_image[i, j, :], c))
                min_b  = min(distances_b)
                min_fb = min(distances_fb)

                if min_b < min_fb:
                    b = np.append(b, [cropped_image[i, j, :]], axis=0)
                    fb = np.delete(fb, np.where(fb==[cropped_image[i,j,:]])[0][0], axis=0)
                    img_mask[i, j] = 0

        #show_images(img_mask)
        cv.imwrite("segmentations/"+img_name+'_'+str(cnt)+'.jpg', img_mask)
    return img_mask

In [7]:
image_path='03.jpg'
gt_image_path='03.png'
gt_image=cv.imread('ground_truth/' + gt_image_path,cv.IMREAD_GRAYSCALE)
img_name=image_path.split('.')[0]
image = cv.imread('images/'+image_path)
r,b_orig,fb_orig,cropped_image=get_regions(image)
# mask=segment_image(cropped_image,5,10,7,b_orig,fb_orig,img_name)
# full_image=np.zeros((image.shape[0],image.shape[1],1))
# full_image[r[1]:r[1]+r[3],r[0]:r[0]+r[2]]=1
# full_image[r[1]:r[1]+r[3],r[0]:r[0]+r[2]]=full_image[r[1]:r[1]+r[3],r[0]:r[0]+r[2]] * mask[:, :, np.newaxis]

print(full_image)
# show_images(full_image)
# cv.imwrite("segmentations/"+img_name+'_classic'+'.jpg', full_image)
# print(measure_performance(full_image.copy(),gt_image.copy()))

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/invokariman/.cache/pypoetry/virtualenvs/ub-g12-compvis-Cpc6amjP-py3.10/lib/python3.10/site-packages/cv2/qt/plugins"


(0, 0, 3)


error: OpenCV(4.7.0) /io/opencv/modules/highgui/src/window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


: 

In [ ]:
# https://www.docs.opencv.org/master/d8/d83/tutorial_py_grabcut.html
def run_grabcut(img):
    mask = np.zeros(img.shape[:2], np.uint8)
    bgdModel = np.zeros((1,65), np.float64)
    fgdModel = np.zeros((1,65), np.float64)
    rect = cv.selectROI(img)
    cv.destroyAllWindows()
    cv.grabCut(img, mask, rect, bgdModel, fgdModel, 5, cv.GC_INIT_WITH_RECT)
    
    mask2 = np.where((mask == 2)|(mask == 0),0,255).astype('uint8')
    return mask2
    
img = cv.imread('images/02.jpg')
gt_image_path='02.png'
gt_image=cv.imread('ground_truth/'+gt_image_path,cv.IMREAD_GRAYSCALE)
mask = run_grabcut(img)
show_images(image_1=mask, name_1='img_grabcut') 
cv.imwrite("segmentations/" + img_name + '_grabcut'+'.jpg', mask)
print(measure_performance(mask,gt_image))

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/grabcut.cpp:386: error: (-215:Assertion failed) !bgdSamples.empty() && !fgdSamples.empty() in function 'initGMMs'
